In [1]:
import time
import os
import copy
from collections import defaultdict

# deep learning/vision libraries
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import cv2 as cv  # OpenCV

# numeric and plotting libraries
import numpy as np
import matplotlib.pyplot as plt
from kard_data_loader import KardDataLoader
import consts as c

In [2]:
train = KardDataLoader('C:\\Users\\bozam\\Desktop\\KARD_all_in_one_folder')
train.getActivityData(c.FILETYPES['screen'])

In [3]:
def normalize(arr):
    return (arr - np.min(arr)) / (np.max(arr) - np.min(arr))

In [4]:
def make_training_set(data):
    generated_images = []
    generated_classes = []
    x = []
    y = []
    z = []
            
    for class_name in range(len(data)):
        image_sequence = []
        for people_sequences in range(len(data[class_name])):
            counter = 0
            for people_coordinates in data[class_name][people_sequences]:
                x.append(people_coordinates[0])
                y.append(people_coordinates[1])
                z.append(people_coordinates[2])
                counter += 1
                
                if counter == 15:
                    counter = 0
                    x = normalize(x)
                    
                    y = normalize(y)
                    
                    z = normalize(z)
                    
                    image_layer = [[x[i], y[i], z[i]] for i in range(15)]
                    generated_images.append(image_layer)
                    generated_classes.append(class_name)
                    x = []
                    y = []
                    z = []
            
    return generated_images, generated_classes

In [5]:
train_images, train_labels = make_training_set(train.data)

In [6]:
print(np.array(train_images).shape)
print(np.array(train_labels).shape)

(63630, 15, 3)
(63630,)


In [11]:
def to_categorical(y, num_classes):
    return np.eye(num_classes, dtype='uint8')[y]

In [143]:
def make_baches(train_data, train_labels, batch_size, stride):
    batch_edges = [[a, a + batch_size] for a in range(0 , len(train_data) - batch_size, stride) if train_labels[a] == train_labels[a + batch_size]]
    
    batch_data = []
    batch_labels = []
    
    for (a, b) in (batch_edges):
        batch_data.append(train_data[a:b])
        batch_labels.append(train_labels[a])
    
    rnd_idx = np.random.permutation(len(batch_data))
    batch_data = np.array(batch_data)
    batch_labels = np.array(batch_labels)
    
    batch_data = batch_data[rnd_idx, :, :, :]
    batch_labels = batch_labels[rnd_idx]
    
    # return (batch_data, batch_labels)
    
    for i in range(len(batch_data)):
        yield (batch_data[i, :, :, :], (batch_labels[i]))

In [144]:
for (batch_data, batch_labels) in make_baches(train_images, train_labels, 30, 2):
    print(batch_data.shape)
    print(batch_labels.shape)

(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3

(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3

()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15

()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15

(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3

...

(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3

()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15

()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15

(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3)
()
(30, 15, 3

KeyboardInterrupt: 

In [117]:
print(batch_data.shape)
print(batch_labels.shape)

(31545, 30, 15, 3)
(31545,)


In [101]:
for (t, l) in make_baches(train_images, train_labels, 40, 3):
    print(t.shape)
    print(l)

(40, 15, 3)
()
14
(40, 15, 3)
()
0
(40, 15, 3)
()
14
(40, 15, 3)
()
2
(40, 15, 3)
()
12
(40, 15, 3)
()
5
(40, 15, 3)
()
10
(40, 15, 3)
()
14
(40, 15, 3)
()
15
(40, 15, 3)
()
8
(40, 15, 3)
()
17
(40, 15, 3)
()
2
(40, 15, 3)
()
10
(40, 15, 3)
()
1
(40, 15, 3)
()
1
(40, 15, 3)
()
0
(40, 15, 3)
()
15
(40, 15, 3)
()
4
(40, 15, 3)
()
6
(40, 15, 3)
()
5
(40, 15, 3)
()
5
(40, 15, 3)
()
8
(40, 15, 3)
()
17
(40, 15, 3)
()
3
(40, 15, 3)
()
17
(40, 15, 3)
()
3
(40, 15, 3)
()
8
(40, 15, 3)
()
9
(40, 15, 3)
()
1
(40, 15, 3)
()
14
(40, 15, 3)
()
17
(40, 15, 3)
()
2
(40, 15, 3)
()
14
(40, 15, 3)
()
4
(40, 15, 3)
()
11
(40, 15, 3)
()
6
(40, 15, 3)
()
3
(40, 15, 3)
()
6
(40, 15, 3)
()
3
(40, 15, 3)
()
13
(40, 15, 3)
()
10
(40, 15, 3)
()
6
(40, 15, 3)
()
11
(40, 15, 3)
()
15
(40, 15, 3)
()
17
(40, 15, 3)
()
17
(40, 15, 3)
()
0
(40, 15, 3)
()
6
(40, 15, 3)
()
0
(40, 15, 3)
()
9
(40, 15, 3)
()
14
(40, 15, 3)
()
11
(40, 15, 3)
()
6
(40, 15, 3)
()
16
(40, 15, 3)
()
13
(40, 15, 3)
()
2
(40, 15, 3)
()
8
(40, 1

0
(40, 15, 3)
()
14
(40, 15, 3)
()
17
(40, 15, 3)
()
4
(40, 15, 3)
()
10
(40, 15, 3)
()
14
(40, 15, 3)
()
16
(40, 15, 3)
()
16
(40, 15, 3)
()
8
(40, 15, 3)
()
17
(40, 15, 3)
()
13
(40, 15, 3)
()
4
(40, 15, 3)
()
2
(40, 15, 3)
()
16
(40, 15, 3)
()
15
(40, 15, 3)
()
6
(40, 15, 3)
()
11
(40, 15, 3)
()
2
(40, 15, 3)
()
3
(40, 15, 3)
()
8
(40, 15, 3)
()
16
(40, 15, 3)
()
16
(40, 15, 3)
()
1
(40, 15, 3)
()
4
(40, 15, 3)
()
10
(40, 15, 3)
()
4
(40, 15, 3)
()
6
(40, 15, 3)
()
1
(40, 15, 3)
()
5
(40, 15, 3)
()
9
(40, 15, 3)
()
14
(40, 15, 3)
()
6
(40, 15, 3)
()
12
(40, 15, 3)
()
3
(40, 15, 3)
()
10
(40, 15, 3)
()
11
(40, 15, 3)
()
11
(40, 15, 3)
()
11
(40, 15, 3)
()
0
(40, 15, 3)
()
9
(40, 15, 3)
()
3
(40, 15, 3)
()
16
(40, 15, 3)
()
13
(40, 15, 3)
()
17
(40, 15, 3)
()
6
(40, 15, 3)
()
11
(40, 15, 3)
()
12
(40, 15, 3)
()
4
(40, 15, 3)
()
11
(40, 15, 3)
()
10
(40, 15, 3)
()
9
(40, 15, 3)
()
4
(40, 15, 3)
()
10
(40, 15, 3)
()
11
(40, 15, 3)
()
13
(40, 15, 3)
()
1
(40, 15, 3)
()
11
(40, 15, 3)
()


15
(40, 15, 3)
()
7
(40, 15, 3)
()
1
(40, 15, 3)
()
14
(40, 15, 3)
()
2
(40, 15, 3)
()
13
(40, 15, 3)
()
11
(40, 15, 3)
()
14
(40, 15, 3)
()
8
(40, 15, 3)
()
12
(40, 15, 3)
()
12
(40, 15, 3)
()
8
(40, 15, 3)
()
13
(40, 15, 3)
()
12
(40, 15, 3)
()
3
(40, 15, 3)
()
2
(40, 15, 3)
()
13
(40, 15, 3)
()
11
(40, 15, 3)
()
10
(40, 15, 3)
()
1
(40, 15, 3)
()
2
(40, 15, 3)
()
12
(40, 15, 3)
()
13
(40, 15, 3)
()
10
(40, 15, 3)
()
15
(40, 15, 3)
()
6
(40, 15, 3)
()
13
(40, 15, 3)
()
0
(40, 15, 3)
()
17
(40, 15, 3)
()
6
(40, 15, 3)
()
0
(40, 15, 3)
()
1
(40, 15, 3)
()
6
(40, 15, 3)
()
4
(40, 15, 3)
()
7
(40, 15, 3)
()
2
(40, 15, 3)
()
8
(40, 15, 3)
()
7
(40, 15, 3)
()
15
(40, 15, 3)
()
17
(40, 15, 3)
()
0
(40, 15, 3)
()
9
(40, 15, 3)
()
13
(40, 15, 3)
()
0
(40, 15, 3)
()
8
(40, 15, 3)
()
14
(40, 15, 3)
()
12
(40, 15, 3)
()
4
(40, 15, 3)
()
4
(40, 15, 3)
()
9
(40, 15, 3)
()
13
(40, 15, 3)
()
12
(40, 15, 3)
()
14
(40, 15, 3)
()
7
(40, 15, 3)
()
7
(40, 15, 3)
()
10
(40, 15, 3)
()
5
(40, 15, 3)
()
4
(4

()
7
(40, 15, 3)
()
3
(40, 15, 3)
()
3
(40, 15, 3)
()
2
(40, 15, 3)
()
11
(40, 15, 3)
()
2
(40, 15, 3)
()
1
(40, 15, 3)
()
8
(40, 15, 3)
()
5
(40, 15, 3)
()
11
(40, 15, 3)
()
3
(40, 15, 3)
()
17
(40, 15, 3)
()
2
(40, 15, 3)
()
1
(40, 15, 3)
()
13
(40, 15, 3)
()
2
(40, 15, 3)
()
13
(40, 15, 3)
()
3
(40, 15, 3)
()
16
(40, 15, 3)
()
11
(40, 15, 3)
()
14
(40, 15, 3)
()
1
(40, 15, 3)
()
16
(40, 15, 3)
()
10
(40, 15, 3)
()
4
(40, 15, 3)
()
5
(40, 15, 3)
()
16
(40, 15, 3)
()
5
(40, 15, 3)
()
4
(40, 15, 3)
()
9
(40, 15, 3)
()
6
(40, 15, 3)
()
5
(40, 15, 3)
()
0
(40, 15, 3)
()
13
(40, 15, 3)
()
1
(40, 15, 3)
()
17
(40, 15, 3)
()
3
(40, 15, 3)
()
10
(40, 15, 3)
()
0
(40, 15, 3)
()
17
(40, 15, 3)
()
3
(40, 15, 3)
()
6
(40, 15, 3)
()
5
(40, 15, 3)
()
1
(40, 15, 3)
()
14
(40, 15, 3)
()
9
(40, 15, 3)
()
13
(40, 15, 3)
()
5
(40, 15, 3)
()
13
(40, 15, 3)
()
11
(40, 15, 3)
()
12
(40, 15, 3)
()
5
(40, 15, 3)
()
2
(40, 15, 3)
()
8
(40, 15, 3)
()
8
(40, 15, 3)
()
9
(40, 15, 3)
()
6
(40, 15, 3)
()
8
(40, 1

()
2
(40, 15, 3)
()
12
(40, 15, 3)
()
13
(40, 15, 3)
()
6
(40, 15, 3)
()
13
(40, 15, 3)
()
12
(40, 15, 3)
()
1
(40, 15, 3)
()
13
(40, 15, 3)
()
11
(40, 15, 3)
()
14
(40, 15, 3)
()
6
(40, 15, 3)
()
7
(40, 15, 3)
()
10
(40, 15, 3)
()
16
(40, 15, 3)
()
13
(40, 15, 3)
()
7
(40, 15, 3)
()
8
(40, 15, 3)
()
11
(40, 15, 3)
()
16
(40, 15, 3)
()
14
(40, 15, 3)
()
9
(40, 15, 3)
()
14
(40, 15, 3)
()
11
(40, 15, 3)
()
5
(40, 15, 3)
()
16
(40, 15, 3)
()
1
(40, 15, 3)
()
6
(40, 15, 3)
()
0
(40, 15, 3)
()
10
(40, 15, 3)
()
14
(40, 15, 3)
()
14
(40, 15, 3)
()
6
(40, 15, 3)
()
13
(40, 15, 3)
()
13
(40, 15, 3)
()
4
(40, 15, 3)
()
4
(40, 15, 3)
()
6
(40, 15, 3)
()
5
(40, 15, 3)
()
14
(40, 15, 3)
()
11
(40, 15, 3)
()
14
(40, 15, 3)
()
0
(40, 15, 3)
()
8
(40, 15, 3)
()
7
(40, 15, 3)
()
1
(40, 15, 3)
()
9
(40, 15, 3)
()
11
(40, 15, 3)
()
0
(40, 15, 3)
()
8
(40, 15, 3)
()
16
(40, 15, 3)
()
5
(40, 15, 3)
()
13
(40, 15, 3)
()
1
(40, 15, 3)
()
14
(40, 15, 3)
()
11
(40, 15, 3)
()
7
(40, 15, 3)
()
13
(40, 15, 3)
(

...

5
(40, 15, 3)
()
0
(40, 15, 3)
()
14
(40, 15, 3)
()
13
(40, 15, 3)
()
4
(40, 15, 3)
()
14
(40, 15, 3)
()
4
(40, 15, 3)
()
15
(40, 15, 3)
()
16
(40, 15, 3)
()
4
(40, 15, 3)
()
8
(40, 15, 3)
()
7
(40, 15, 3)
()
10
(40, 15, 3)
()
12
(40, 15, 3)
()
0
(40, 15, 3)
()
16
(40, 15, 3)
()
5
(40, 15, 3)
()
15
(40, 15, 3)
()
8
(40, 15, 3)
()
0
(40, 15, 3)
()
0
(40, 15, 3)
()
8
(40, 15, 3)
()
8
(40, 15, 3)
()
15
(40, 15, 3)
()
17
(40, 15, 3)
()
3
(40, 15, 3)
()
16
(40, 15, 3)
()
13
(40, 15, 3)
()
12
(40, 15, 3)
()
16
(40, 15, 3)
()
13
(40, 15, 3)
()
3
(40, 15, 3)
()
13
(40, 15, 3)
()
12
(40, 15, 3)
()
13
(40, 15, 3)
()
15
(40, 15, 3)
()
16
(40, 15, 3)
()
2
(40, 15, 3)
()
10
(40, 15, 3)
()
1
(40, 15, 3)
()
6
(40, 15, 3)
()
10
(40, 15, 3)
()
13
(40, 15, 3)
()
1
(40, 15, 3)
()
8
(40, 15, 3)
()
16
(40, 15, 3)
()
3
(40, 15, 3)
()
0
(40, 15, 3)
()
9
(40, 15, 3)
()
15
(40, 15, 3)
()
2
(40, 15, 3)
()
10
(40, 15, 3)
()
10
(40, 15, 3)
()
3
(40, 15, 3)
()
1
(40, 15, 3)
()
4
(40, 15, 3)
()
7
(40, 15, 3)
()
2
(

(40, 15, 3)
()
13
(40, 15, 3)
()
13
(40, 15, 3)
()
7
(40, 15, 3)
()
13
(40, 15, 3)
()
13
(40, 15, 3)
()
13
(40, 15, 3)
()
15
(40, 15, 3)
()
1
(40, 15, 3)
()
13
(40, 15, 3)
()
10
(40, 15, 3)
()
17
(40, 15, 3)
()
13
(40, 15, 3)
()
6
(40, 15, 3)
()
14
(40, 15, 3)
()
10
(40, 15, 3)
()
10
(40, 15, 3)
()
11
(40, 15, 3)
()
1
(40, 15, 3)
()
10
(40, 15, 3)
()
6
(40, 15, 3)
()
2
(40, 15, 3)
()
13
(40, 15, 3)
()
1
(40, 15, 3)
()
11
(40, 15, 3)
()
4
(40, 15, 3)
()
10
(40, 15, 3)
()
13
(40, 15, 3)
()
5
(40, 15, 3)
()
6
(40, 15, 3)
()
3
(40, 15, 3)
()
12
(40, 15, 3)
()
4
(40, 15, 3)
()
8
(40, 15, 3)
()
0
(40, 15, 3)
()
7
(40, 15, 3)
()
10
(40, 15, 3)
()
3
(40, 15, 3)
()
12
(40, 15, 3)
()
14
(40, 15, 3)
()
16
(40, 15, 3)
()
1
(40, 15, 3)
()
15
(40, 15, 3)
()
10
(40, 15, 3)
()
13
(40, 15, 3)
()
5
(40, 15, 3)
()
13
(40, 15, 3)
()
6
(40, 15, 3)
()
3
(40, 15, 3)
()
1
(40, 15, 3)
()
17
(40, 15, 3)
()
7
(40, 15, 3)
()
0
(40, 15, 3)
()
2
(40, 15, 3)
()
1
(40, 15, 3)
()
11
(40, 15, 3)
()
15
(40, 15, 3)
()
1


(40, 15, 3)
()
3
(40, 15, 3)
()
4
(40, 15, 3)
()
3
(40, 15, 3)
()
13
(40, 15, 3)
()
16
(40, 15, 3)
()
6
(40, 15, 3)
()
15
(40, 15, 3)
()
10
(40, 15, 3)
()
0
(40, 15, 3)
()
0
(40, 15, 3)
()
10
(40, 15, 3)
()
1
(40, 15, 3)
()
9
(40, 15, 3)
()
8
(40, 15, 3)
()
10
(40, 15, 3)
()
15
(40, 15, 3)
()
7
(40, 15, 3)
()
13
(40, 15, 3)
()
2
(40, 15, 3)
()
13
(40, 15, 3)
()
1
(40, 15, 3)
()
15
(40, 15, 3)
()
5
(40, 15, 3)
()
4
(40, 15, 3)
()
11
(40, 15, 3)
()
14
(40, 15, 3)
()
12
(40, 15, 3)
()
1
(40, 15, 3)
()
10
(40, 15, 3)
()
1
(40, 15, 3)
()
5
(40, 15, 3)
()
9
(40, 15, 3)
()
2
(40, 15, 3)
()
12
(40, 15, 3)
()
0
(40, 15, 3)
()
17
(40, 15, 3)
()
1
(40, 15, 3)
()
10
(40, 15, 3)
()
13
(40, 15, 3)
()
14
(40, 15, 3)
()
10
(40, 15, 3)
()
5
(40, 15, 3)
()
17
(40, 15, 3)
()
3
(40, 15, 3)
()
2
(40, 15, 3)
()
5
(40, 15, 3)
()
8
(40, 15, 3)
()
8
(40, 15, 3)
()
11
(40, 15, 3)
()
6
(40, 15, 3)
()
14
(40, 15, 3)
()
13
(40, 15, 3)
()
5
(40, 15, 3)
()
4
(40, 15, 3)
()
1
(40, 15, 3)
()
15
(40, 15, 3)
()
7
(40, 1

()
4
(40, 15, 3)
()
5
(40, 15, 3)
()
9
(40, 15, 3)
()
3
(40, 15, 3)
()
14
(40, 15, 3)
()
13
(40, 15, 3)
()
13
(40, 15, 3)
()
4
(40, 15, 3)
()
14
(40, 15, 3)
()
2
(40, 15, 3)
()
9
(40, 15, 3)
()
16
(40, 15, 3)
()
10
(40, 15, 3)
()
13
(40, 15, 3)
()
11
(40, 15, 3)
()
13
(40, 15, 3)
()
14
(40, 15, 3)
()
3
(40, 15, 3)
()
11
(40, 15, 3)
()
13
(40, 15, 3)
()
8
(40, 15, 3)
()
10
(40, 15, 3)
()
10
(40, 15, 3)
()
13
(40, 15, 3)
()
15
(40, 15, 3)
()
12
(40, 15, 3)
()
0
(40, 15, 3)
()
6
(40, 15, 3)
()
5
(40, 15, 3)
()
14
(40, 15, 3)
()
17
(40, 15, 3)
()
17
(40, 15, 3)
()
14
(40, 15, 3)
()
10
(40, 15, 3)
()
13
(40, 15, 3)
()
2
(40, 15, 3)
()
11
(40, 15, 3)
()
13
(40, 15, 3)
()
6
(40, 15, 3)
()
6
(40, 15, 3)
()
6
(40, 15, 3)
()
4
(40, 15, 3)
()
5
(40, 15, 3)
()
5
(40, 15, 3)
()
3
(40, 15, 3)
()
8
(40, 15, 3)
()
8
(40, 15, 3)
()
2
(40, 15, 3)
()
2
(40, 15, 3)
()
0
(40, 15, 3)
()
17
(40, 15, 3)
()
12
(40, 15, 3)
()
15
(40, 15, 3)
()
13
(40, 15, 3)
()
3
(40, 15, 3)
()
14
(40, 15, 3)
()
3
(40, 15, 3)
(

5
(40, 15, 3)
()
7
(40, 15, 3)
()
14
(40, 15, 3)
()
17
(40, 15, 3)
()
16
(40, 15, 3)
()
8
(40, 15, 3)
()
10
(40, 15, 3)
()
13
(40, 15, 3)
()
14
(40, 15, 3)
()
2
(40, 15, 3)
()
13
(40, 15, 3)
()
15
(40, 15, 3)
()
16
(40, 15, 3)
()
8
(40, 15, 3)
()
2
(40, 15, 3)
()
10
(40, 15, 3)
()
6
(40, 15, 3)
()
17
(40, 15, 3)
()
10
(40, 15, 3)
()
6
(40, 15, 3)
()
0
(40, 15, 3)
()
13
(40, 15, 3)
()
9
(40, 15, 3)
()
10
(40, 15, 3)
()
14
(40, 15, 3)
()
3
(40, 15, 3)
()
2
(40, 15, 3)
()
13
(40, 15, 3)
()
1
(40, 15, 3)
()
4
(40, 15, 3)
()
0
(40, 15, 3)
()
12
(40, 15, 3)
()
14
(40, 15, 3)
()
15
(40, 15, 3)
()
0
(40, 15, 3)
()
9
(40, 15, 3)
()
6
(40, 15, 3)
()
0
(40, 15, 3)
()
14
(40, 15, 3)
()
0
(40, 15, 3)
()
5
(40, 15, 3)
()
3
(40, 15, 3)
()
16
(40, 15, 3)
()
1
(40, 15, 3)
()
13
(40, 15, 3)
()
16
(40, 15, 3)
()
17
(40, 15, 3)
()
0
(40, 15, 3)
()
3
(40, 15, 3)
()
7
(40, 15, 3)
()
8
(40, 15, 3)
()
10
(40, 15, 3)
()
14
(40, 15, 3)
()
14
(40, 15, 3)
()
13
(40, 15, 3)
()
2
(40, 15, 3)
()
5
(40, 15, 3)
()
7
(

In [129]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [130]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [155]:
def train_model(model, criterion, optimizer, num_epochs=100):
    
    metrics = defaultdict(list)
    phase = "train"
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch}/{num_epochs - 1}')
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train']:
            if phase == 'train':
                model.train()  # Set model to training mode
#             else:
#                 model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            # 40, 15, 3 - > 3, 15, 40
            #print(np.array(train_images).shape)
            #print(np.array(train_labels).shape)
            for (inputs, labels) in make_baches(train_images, train_labels, 40, 3):
                #print(inputs)
                #print(labels)
                inputs = inputs.reshape(3, 15, 40)
                inputs = inputs.reshape(-1, 3, 15, 40)
                inputs = torch.from_numpy(inputs)
                labels = torch.from_numpy(np.array([labels]))
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward
                # track history if only in train
                #print("Pre setovanja gradienata!")
                with torch.set_grad_enabled(True):
                    #print("Ulazim u setovanje gradienata!!!!")
                    outputs = model(inputs.float())
                    
                    #outputs = np.eye(18)[2]
                    
                    _, preds = torch.max(outputs, 1)
                    #print(outputs)
                    
                    #print(labels)
                    loss = criterion(outputs, labels.long())
                    #print(loss)
                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data).item()
            
            epoch_loss = running_loss / 31545
            epoch_acc = float(running_corrects) / 31545

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
            
            metrics[phase+"_loss"].append(epoch_loss)
            metrics[phase+"_acc"].append(epoch_acc)

In [ ]:
class CustomCNN(nn.Module):
    # [50, 100, 3, 3], but got 3-dimensional input of size [40, 15, 3]
    def __init__(self):
        super().__init__()
        
        num_of_channels = [3, 16, 32, 64, 128]
        kernel_sizes = [3, 3, 3, 3]
        stride_sizes = [2, 2, 2, 2]
        
        self.relu = torch.nn.ReLU()
        
        self.conv1 = nn.Conv2d(num_of_channels[0], num_of_channels[1], kernel_size=(4, 1), stride=stride_sizes[0])
        self.conv2 = nn.Conv2d(num_of_channels[1], num_of_channels[2], kernel_size=(2, 9), stride=stride_sizes[1])
        self.conv3 = nn.Conv2d(num_of_channels[2], num_of_channels[3], kernel_size=(1, 2), stride=stride_sizes[2])
        self.conv4 = nn.Conv2d(num_of_channels[3], num_of_channels[4], kernel_size=(2, 3), stride=stride_sizes[3])
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = nn.Linear(num_of_channels[-1] * 7 * 7, 18)
        
        
    def forward(self, x):
        y = self.relu(self.conv1(x))
        y = self.relu(self.conv2(y))
        y = self.relu(self.conv3(y))
        y = self.relu(self.conv4(y))
        y = self.avgpool(y)
        y = torch.flatten(y, 1)
        y = self.classifier(y)
        return y
    
    
custom_cnn = CustomCNN().to(device)
optimizer_conv = optim.Adam(filter(lambda p: p.requires_grad, custom_cnn.parameters()))

print(f"number of params in model {count_parameters(custom_cnn)}")

#
# todo: (#11) Why is acc 0.5? Anybody?
#
criterion = nn.CrossEntropyLoss()
model_conv = train_model(custom_cnn, criterion, optimizer_conv, num_epochs=25)

plt.subplots_adjust(wspace=1, hspace=1)
fig, ax = plt.subplots(nrows=2, ncols=2)
keys = list(metrics.keys())
cnt = 0
for row in ax:
    for col in row:
        col.set_title(keys[cnt])
        col.plot(metrics[keys[cnt]])
        cnt += 1

number of params in model 175810
Epoch 0/24
----------
train Loss: 0.7285 Acc: 0.4162
Epoch 1/24
----------
train Loss: 0.4393 Acc: 0.5146
Epoch 2/24
----------
train Loss: 0.4185 Acc: 0.5278
Epoch 3/24
----------
train Loss: 0.4080 Acc: 0.5309
Epoch 4/24
----------
train Loss: 0.4054 Acc: 0.5352
Epoch 5/24
----------
train Loss: 0.4070 Acc: 0.5368
Epoch 6/24
----------
train Loss: 0.4077 Acc: 0.5377
Epoch 7/24
----------
train Loss: 0.4074 Acc: 0.5407
Epoch 8/24
----------
train Loss: 0.4280 Acc: 0.5345
Epoch 9/24
----------
train Loss: 0.4240 Acc: 0.5350
Epoch 10/24
----------
train Loss: 0.4339 Acc: 0.5326
Epoch 11/24
----------
train Loss: 0.4495 Acc: 0.5337
Epoch 12/24
----------
train Loss: 0.4619 Acc: 0.5317
Epoch 13/24
----------
train Loss: 0.4812 Acc: 0.5288
Epoch 14/24
----------
